In [1]:
import pybamm

## CC-CV

In [102]:
experiment = pybamm.Experiment(
    [
        (
            "Charge at 5 C until 4.2 V",
            "Hold at 4.2 V until 0.1A",
        ),
    ]
)
model = pybamm.lithium_ion.DFN(name="CC-CV")
model.variables["Plating overpotential [V]"] = pybamm.min(
    model.variables["Negative electrode surface potential difference [V]"]
)
param = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)

solver = pybamm.CasadiSolver(dt_max=1)#extra_options_setup={"max_num_steps": 200}, )
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=param, solver=solver)
sim.solve()

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.


## CC-Ceta

In [103]:
    
def constant_current_constant_overpotential(variables):
    I = variables["Current [A]"]
    s_I = pybamm.InputParameter("Current switch")
    eta_pl = pybamm.boundary_value(
        variables["Negative electrode surface potential difference [V]"], "right"
    )
    s_eta = pybamm.InputParameter("Overpotential switch")
    V = variables["Terminal voltage [V]"]
    s_V = pybamm.InputParameter("Voltage switch")

    return (
        s_I * (I - -5*5)
        + s_eta * (eta_pl)
        + s_V * (V - 4.2)
    )

In [106]:
model2 = pybamm.lithium_ion.DFN({"operating mode": constant_current_constant_overpotential}, name="CC-Ceta-CV")

# add termination event for li plating
eta_pl = model2.variables["Negative electrode surface potential difference [V]"]
model2.events.append(
    pybamm.Event("Li plating [experiment]", 
    pybamm.min(eta_pl) - pybamm.InputParameter("Plating cut-off [V]"))
)
model2.events.append(
    pybamm.Event(
        "Current cut-off (positive) [A] [experiment]",
        model2.variables["Current [A]"]
        + abs(pybamm.InputParameter("Current cut-off [A]")),
    )
)
model2.events.append(
    pybamm.Event(
        "Voltage cut-off [V] [experiment]",
        model2.variables["Terminal voltage [V]"]
        - pybamm.InputParameter("Voltage cut-off [V]"),
    )
)
model2.variables["Plating overpotential [V]"] = pybamm.min(
    model2.variables["Negative electrode surface potential difference [V]"]
)

param2 = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
param2["Upper voltage cut-off [V]"] = 5
param2["Current function [A]"] = -5 * 5
solver = pybamm.CasadiSolver(dt_max = 1, extra_options_setup={"max_num_steps": 200})

sim2 = pybamm.Simulation(model2, parameter_values=param2, solver=solver)

# Re-initialize solution, e.g. for solving multiple times with different
# inputs without having to build the simulation again
sim2._solution = None
# Step through all experimental conditions

experiment_inputs = [
    {"Current switch": 1, "Overpotential switch": 0, "Voltage switch": 0,
     "Plating cut-off [V]": 0, "Current cut-off [A]": -1e10, "Voltage cut-off [V]": 1e10},
    {"Current switch": 0, "Overpotential switch": 1, "Voltage switch": 0,
     "Plating cut-off [V]": 1e10, "Current cut-off [A]": -1e10, "Voltage cut-off [V]": 4.2},
    {"Current switch": 0, "Overpotential switch": 0, "Voltage switch": 1,
     "Plating cut-off [V]": 1e10, "Current cut-off [A]": 0.1, "Voltage cut-off [V]": 1e10},
]
experiment_times = [3600/5, 3600,3600]
period = 60

for idx, (exp_inputs, dt) in enumerate(
    zip(experiment_inputs, experiment_times)
):
    # Make sure we take at least 2 timesteps
    npts = max(int(round(dt / period)) + 1, 2)
    sim2.step(dt, solver=solver, npts=npts, inputs=exp_inputs)
    # Only allow events specified by experiment
    if not (
        sim2._solution.termination == "final time"
        or "[experiment]" in sim2._solution.termination
    ):
        pybamm.logger.warning(
            "\n\n\tExperiment is infeasible: '{}' ".format(
                sim2._solution.termination
            )
            + "was triggered during '{}'. ".format(
                exp_inputs
            )
            + "Try reducing current, shortening the time interval, "
            "or reducing the period.\n\n"
        )
        break

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.


## Plot

In [109]:
plot = pybamm.dynamic_plot(
    [sim,sim2], 
    ["Current [A]", "Terminal voltage [V]", "Plating overpotential [V]"]
)
plot.fig.savefig("Ceta_charging.png")

interactive(children=(FloatSlider(value=0.0, description='t', max=2101.9634413253807, step=21.019634413253808)…

## No CV

In [112]:
model2 = pybamm.lithium_ion.DFN({"operating mode": constant_current_constant_overpotential}, name="CC-Ceta")

# add termination event for li plating
eta_pl = model2.variables["Negative electrode surface potential difference [V]"]
model2.events.append(
    pybamm.Event("Li plating [experiment]", 
    pybamm.min(eta_pl) - pybamm.InputParameter("Plating cut-off [V]"))
)
model2.events.append(
    pybamm.Event(
        "Current cut-off (positive) [A] [experiment]",
        model2.variables["Current [A]"]
        + abs(pybamm.InputParameter("Current cut-off [A]")),
    )
)
model2.events.append(
    pybamm.Event(
        "Voltage cut-off [V] [experiment]",
        model2.variables["Terminal voltage [V]"]
        - pybamm.InputParameter("Voltage cut-off [V]"),
    )
)
model2.variables["Plating overpotential [V]"] = pybamm.min(
    model2.variables["Negative electrode surface potential difference [V]"]
)

param2 = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
param2["Upper voltage cut-off [V]"] = 5
param2["Current function [A]"] = -5 * 5
solver = pybamm.CasadiSolver(dt_max = 1, extra_options_setup={"max_num_steps": 200})

sim2 = pybamm.Simulation(model2, parameter_values=param2, solver=solver)

# Re-initialize solution, e.g. for solving multiple times with different
# inputs without having to build the simulation again
sim2._solution = None
# Step through all experimental conditions

experiment_inputs = [
    {"Current switch": 1, "Overpotential switch": 0, "Voltage switch": 0,
     "Plating cut-off [V]": 0, "Current cut-off [A]": -1e10, "Voltage cut-off [V]": 1e10},
    {"Current switch": 0, "Overpotential switch": 1, "Voltage switch": 0,
     "Plating cut-off [V]": 1e10, "Current cut-off [A]": 0.1, "Voltage cut-off [V]": 1e10},
]
experiment_times = [3600/5, 3600]
period = 60

for idx, (exp_inputs, dt) in enumerate(
    zip(experiment_inputs, experiment_times)
):
    # Make sure we take at least 2 timesteps
    npts = max(int(round(dt / period)) + 1, 2)
    sim2.step(dt, solver=solver, npts=npts, inputs=exp_inputs)
    # Only allow events specified by experiment
    if not (
        sim2._solution.termination == "final time"
        or "[experiment]" in sim2._solution.termination
    ):
        pybamm.logger.warning(
            "\n\n\tExperiment is infeasible: '{}' ".format(
                sim2._solution.termination
            )
            + "was triggered during '{}'. ".format(
                exp_inputs
            )
            + "Try reducing current, shortening the time interval, "
            "or reducing the period.\n\n"
        )
        break

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
At t = 0.0535793, , mxstep steps taken before reaching tout.
2021-01-14 16:44:37,548 - [WARNING] <ipython-input-112-5ad2235ed7f6>.<module>(59): 

	Experiment is infeasible: 'event: Minumum positive particle surface concentration' was triggered during '{'Current switch': 0, 'Overpotential switch': 1, 'Voltage switch': 0, 'Plating cut-off [V]': 10000000000.0, 'Current cut-off [A]': 0.1, 'Voltage cut-off [V]': 10000000000.0}'. Try reducing current, shortening the time interval, or reducing the period.




In [113]:
plot = pybamm.dynamic_plot(
    [sim,sim2], 
    ["Current [A]", "Terminal voltage [V]", "Plating overpotential [V]"]
)
plot.fig.savefig("Ceta_charging2.png")

interactive(children=(FloatSlider(value=0.0, description='t', max=2044.9533223661676, step=20.449533223661675)…